In [ ]:
!./build.sh

In [ ]:
!./format.sh
!echo "=== Testing Removing Unused Import and Varialbe ==="
!autoflake --in-place --remove-unused-variables --remove-all-unused-imports get_german_doc.py

In [1]:
import os

from deploykit import (
    build_package,
    upload_package, 
    deploy_pacakge, 
    update_package_code,
    update_env_vars
)

2022-12-12 02:31:56 [INFO] botocore.credentials - Found credentials in shared credentials file: ~/.aws/credentials


# Test Lambda

In [2]:
from get_german_doc import handler

2022-12-12 02:31:58 [INFO] botocore.credentials - Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
# A failed Test
test_event = {"queryStringParameters": {}}
response = handler(event=test_event, context=_)
response

{'statusCode': 403,
 'body': '{"statusCode": 403, "data": {}, "errorMessage": {"errorType": "ParameterError", "httpStatus": 403, "trace": "Following field are requered to query the api: pid, version"}}'}

In [5]:
# Test A successfull response
test_event = {"queryStringParameters": {"pid": "test-01-2022-12-07T18:12:35", "version": "latest"}}
response = handler(event=test_event, context=_)

In [6]:
assert response['statusCode'] == 200

# Deploy Lambda Handler

In [ ]:
# !pip install --upgrade -r requirements.txt --target ./packages

In [ ]:
BUCKET = os.environ['BUCKET']
BUCKET_KEY = "lambdas/get-german-doc.zip"
FUNCTION_NAME = "quack-tsln-get-german-doc"

ENV_DYNAMODB = os.environ["DB_NAME"]
ENV_VARIABLES = {
    'Variables': {
        'BUCKET': BUCKET, 
        'DB_NAME': ENV_DYNAMODB, 
        "LAMBDA_RUNTIME_ENV": "cloud"
    }
}
CODE_INFO = {"S3Bucket": BUCKET, "S3Key": BUCKET_KEY}
PACKAGE = BUCKET_KEY.replace("lambdas/", "")

In [ ]:
build_package(PACKAGE)

In [ ]:
upload_package(PACKAGE, BUCKET, BUCKET_KEY)

In [ ]:
DESC = "Retrive the German translated file from s3 for a project"
MAIN_PYFILE = "get_german_doc"

In [ ]:
try:
    deploy_pacakge(
        function_name=FUNCTION_NAME, 
        handler=MAIN_PYFILE,
        timeout=180, 
        desc=DESC, 
        env_vars=ENV_VARIABLES,
        code_info=CODE_INFO,
        memory_size=256
    )
except Exception as e:
    print(e)
    update_package_code(function_name=FUNCTION_NAME, bucket=BUCKET, key=BUCKET_KEY)

In [ ]:
update_env_vars(function_name=FUNCTION_NAME, envvars=ENV_VARIABLES)

In [ ]:
# Create Query Parameter
# "&".join([f"{key}={value}" for key, value in test_event["queryStringParameters"].items()])